## 1. Environment Setup and Dependencies

First, let's load our environment variables and import the required libraries. The `.env` file contains configuration settings that we'll use throughout this notebook.

In [56]:
# Install required packages (run this if packages are not already installed)
%pip install python-dotenv requests lxml matplotlib drawsvg

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [57]:
# Import all required libraries
import os
import datetime
import json
import requests
import dotenv
from lxml import html
import matplotlib.pyplot as plt
import drawsvg as draw
import re

# Import our custom utilities
from scraping_utils import get_url, parse

print("All libraries imported successfully!")

All libraries imported successfully!


In [58]:
# Load environment variables from .env file (force-reload and map new names to legacy names)
from dotenv import dotenv_values, load_dotenv
# Try to override existing env vars when supported
try:
    load_dotenv(override=True)
except TypeError:
    for k, v in dotenv_values('.env').items():
        if v is not None:
            os.environ[k] = v

# Backwards compatibility: if the notebook expects YEAR/URL/etc but we have RAINFALL_URL/START_YEAR, map them
# Map only when legacy names are not set to avoid overwriting explicit values
if os.getenv('YEAR') is None and os.getenv('START_YEAR') is not None:
    os.environ['YEAR'] = os.getenv('START_YEAR')
if os.getenv('URL') is None and os.getenv('RAINFALL_URL') is not None:
    # Keep legacy URL format using YEAR placeholder if needed
    os.environ['URL'] = os.getenv('RAINFALL_URL')
# ROW_XPATH/COL_XPATH may not be needed for the CSV flow; leave as-is unless provided
# MULTICITY_URL fallback: map from MULTICITY_URL or leave None
if os.getenv('MULTICITY_URL') is None and os.getenv('MULTICITY_URL') is None:
    pass

# Display the loaded and mapped environment variables
print("Loaded environment variables (after mapping):")
print(f"YEAR: {os.getenv('YEAR')}")
print(f"URL: {os.getenv('URL')}")
print(f"ROW_XPATH: {os.getenv('ROW_XPATH')}")
print(f"COL_XPATH: {os.getenv('COL_XPATH')}")
print(f"MULTICITY_URL: {os.getenv('MULTICITY_URL')}")
print(f"RAINFALL_URL: {os.getenv('RAINFALL_URL')}")
print(f"RAINFALL_STATION_NAME: {os.getenv('RAINFALL_STATION_NAME')}")
print(f"START_YEAR: {os.getenv('START_YEAR')}")
print(f"END_YEAR: {os.getenv('END_YEAR')}")

Loaded environment variables (after mapping):
YEAR: 2010
URL: https://data.weather.gov.hk/weatherAPI/cis/csvfile/SE/ALL/daily_SE_RF_ALL.csv
ROW_XPATH: None
COL_XPATH: None
MULTICITY_URL: None
RAINFALL_URL: https://data.weather.gov.hk/weatherAPI/cis/csvfile/SE/ALL/daily_SE_RF_ALL.csv
RAINFALL_STATION_NAME: Kaitak
START_YEAR: 2010
END_YEAR: 2025


## 2. Web Scraping Utilities

Let's examine and demonstrate the utility functions we've created for web scraping. These functions help us fetch web pages and parse different data formats.

In [59]:
# Let's look at our scraping utilities
import inspect

print("get_url function:")
print(inspect.getsource(get_url))
print("\n" + "="*50 + "\n")
print("parse function:")
print(inspect.getsource(parse))

get_url function:
def get_url(url, filename):
    if not os.path.exists(filename):
        
        # set useragent for requests
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        
        # fetch the page if it doesn't exist
        page = requests.get(url, headers=headers)

        # save the page to a file
        with open(filename, 'w', encoding='UTF8') as f:
            f.write(page.text)

        page = page.text

    else:
        # if the page exists, read it from the file
        with open(filename, 'r', encoding='UTF8') as f:
            page = f.read() 
            
    return page



parse function:
def parse(page, mode = 'html'):
    match mode:
        case 'html':
            return html.fromstring(page)
        case 'json':
            return json.loads(page)



In [60]:
# Demonstrate the get_url function with a simple example
# This will either fetch from the web or read from cache

test_url = "https://httpbin.org/json"
test_filename = "test_page.html"

print("Fetching test page...")
page_content = get_url(test_url, test_filename)
print(f"Content length: {len(page_content)} characters")
print(f"First 200 characters: {page_content[:200]}...")

Fetching test page...
Content length: 429 characters
First 200 characters: {
  "slideshow": {
    "author": "Yours Truly", 
    "date": "date of publication", 
    "slides": [
      {
        "title": "Wake up to WonderWidgets!", 
        "type": "all"
      }, 
      {
    ...


In [61]:
# Demonstrate parsing JSON data
json_data = parse(page_content, 'json')
print("Parsed JSON data:")
print(json.dumps(json_data, indent=2))

Parsed JSON data:
{
  "slideshow": {
    "author": "Yours Truly",
    "date": "date of publication",
    "slides": [
      {
        "title": "Wake up to WonderWidgets!",
        "type": "all"
      },
      {
        "items": [
          "Why <em>WonderWidgets</em> are great",
          "Who <em>buys</em> WonderWidgets"
        ],
        "title": "Overview",
        "type": "all"
      }
    ],
    "title": "Sample Slide Show"
  }
}


## 3. HTML Data Extraction and Processing

Now let's work with the actual HTML data that was scraped from the Hong Kong Observatory rainfall information.

In [ ]:
# Load and examine the crawled HTML page
# Prefer a local processed CSV if available — this skips HTML parsing when you already have data
csv_file = 'rainfall_processed.csv'
data = []
if os.path.exists(csv_file):
    print(f'Local CSV found: {csv_file} — loading and skipping HTML parsing')
    import csv
    with open(csv_file, 'r', encoding='utf8') as f:
        rdr = csv.DictReader(f)
        for row in rdr:
            # Expecting columns: datetime, rainfall_mm, humidity_pct
            try:
                dt = datetime.datetime.strptime(row.get('datetime','')[:16], '%Y-%m-%d %H:%M')
            except Exception:
                continue
            rainfall = None
            humidity = None
            if row.get('rainfall_mm') not in (None, ''):
                try:
                    rainfall = float(row.get('rainfall_mm'))
                except Exception:
                    rainfall = None
            if row.get('humidity_pct') not in (None, ''):
                try:
                    humidity = float(row.get('humidity_pct'))
                except Exception:
                    humidity = None
            data.append((dt, rainfall, humidity))
    print(f'Loaded {len(data)} records from {csv_file}')
    # Provide downstream variables expected by later cells
    html_content = None
    tree = None
    rows = []
else:
    # Load and examine the crawled HTML page
    year = int(os.getenv('YEAR', 2024))
    filename = os.getenv('FILENAME', "crawled-page-{year}.html").format(year=year)

    print(f"Loading HTML file: {filename}")

    # Check if the file exists
    if os.path.exists(filename):
        with open(filename, 'r', encoding='UTF8') as f:
            html_content = f.read()
    
        print(f"HTML file loaded successfully!")
        print(f"Content length: {len(html_content)} characters")
        print(f"First 500 characters:\n{html_content[:500]}...")
    else:
        print(f"HTML file {filename} not found. Let's fetch it from the web.")
        
        # Use environment variable URL and YEAR for Hong Kong rainfall 2024
        url = os.getenv('URL').replace('${YEAR}', str(year))
        print(f"Fetching from: {url}")
        
        html_content = get_url(url, filename)
        print(f"Content fetched and saved to {filename}")

Loading HTML file: crawled-page-2010.html
HTML file loaded successfully!
Content length: 91679 characters
First 500 characters:
ï»¿ç¸½é¨é (æ¯«ç±³) - åå¾·
Total Rainfall (mm) - Kai Tak
å¹´/Year,æ/Month,æ¥/Day,æ¸å¼/Value,æ¸æå®æ´æ§/data Completeness
2010,1,1,***,
2010,1,2,***,
2010,1,3,***,
2010,1,4,***,
2010,1,5,***,
2010,1,6,***,
2010,1,7,***,
2010,1,8,***,
2010,1,9,***,
2010,1,10,***,
2010,1,11,***,
2010,1,12,***,
2010,1,13,***,
2010,1,14,***,
2010,1,15,***,
2010,1,16,***,
2010,1,17,***,
2010,1,18,***,
2010,1,19,***,
2010,1,20,***,
2010,1,21,***,
2010,1,22,***,
2010,1,23,***,
2010,1,24,***,
2010,1,25,***,...


In [63]:
# Parse the HTML content
tree = parse(html_content, 'html')

# Extract table structure using XPath
# Use fallback XPath if environment variables are not set
row_xpath = os.getenv('ROW_XPATH')
col_xpath = os.getenv('COL_XPATH')
if row_xpath is None:
    # Fallback: select all <tr> elements
    row_xpath = '//tr'
if col_xpath is None:
    # Fallback: select all <td> and <th> elements within a row
    col_xpath = './td | ./th'

rows = tree.xpath(row_xpath)
print(f"Found {len(rows)} rows in the HTML table")

# Examine the first few rows
print("\nFirst 3 rows structure:")
for i, row in enumerate(rows[:3]):
    columns = row.xpath(os.getenv('COL_XPATH'))
    column_texts = [col.text_content().strip() for col in columns]
    print(f"Row {i+1}: {column_texts}")

Found 0 rows in the HTML table

First 3 rows structure:


## 4. CSV Data Handling for Rainfall

Let's process the rainfall data and create a CSV file, then load and analyze the data.

In [ ]:
# Process rainfall and humidity data
# We'll attempt to extract both rainfall (mm) and humidity (%) when available
data = []  # list of tuples: (datetime, rainfall_mm_or_None, humidity_pct_or_None)
row_num = 0

print("Processing rainfall and humidity data...")

for row in tree.xpath(row_xpath):
    columns = row.xpath(col_xpath)
    columns = [column.text_content().strip() if column.text_content() is not None else '' for column in columns]
    row_string = " \
,
,
,
":
        continue
    
    row_num += 1
    
    # Only log first few rows for demonstration
    if row_num <= 5:
        print(f'Row {row_num}: {row_string}')
    
    # Skip header or invalid rows
    if len(columns) < 3 or not columns[0].lstrip('0').isdigit():
        continue
    
    try:
        month = int(columns[0])
        day = int(columns[1])
        
        # Iterate through the remaining columns trying to pair time, rainfall and possibly humidity
        # Common layouts: [MM, DD, HHMM, mm, HHMM, mm, ...] or sometimes humidity appears nearby like '45%']
        for i in range(2, len(columns)):
            col = columns[i]
            # Identify time-like entries (HHMM or HMM)
            time_digits = re.sub(r'[^0-9]', '', col)
            if len(time_digits) >= 3 and len(time_digits) <= 4:
                # Normalize to HHMM if needed
                if len(time_digits) == 3:
                    time_digits = '0' + time_digits
                if not time_digits.isdigit():
                    continue
                hour = int(time_digits[:2])
                minute = int(time_digits[2:])
                # Build datetime, guard against invalid hours/minutes
                try:
                    dt = datetime.datetime(year, month, day, hour, minute)
                except ValueError:
                    continue
                # Attempt to read next columns for rainfall and humidity
                rainfall = None
                humidity = None
                # Next column often contains rainfall amount
                if i+1 < len(columns):
                    next_col = columns[i+1]
                    # Extract numeric rainfall value (allow decimal)
                    val = re.sub(r'[^0-9.]', '', next_col)
                    if val != '':
                        try:
                            rainfall = float(val)
                        except ValueError:
                            rainfall = None
                # Look ahead one more column for humidity if present (e.g., '45%')
                if i+2 < len(columns):
                    maybe_hum = columns[i+2]
                    hum_digits = re.sub(r'[^0-9.]', '', maybe_hum)
                    if hum_digits != '':
                        try:
                            humidity = float(hum_digits)
                        except ValueError:
                            humidity = None
                # As a fallback, search the same row for any value containing '%'
                if humidity is None:
                    for c in columns:
                        if '%' in c:
                            h = re.sub(r'[^0-9.]', '', c)
                            if h != '':
                                try:
                                    humidity = float(h)
                                    break
                                except ValueError:
                                    continue
                # Append record only if we have at least rainfall or humidity
                if rainfall is not None or humidity is not None:
                    data.append((dt, rainfall, humidity))
                    if len(data) <= 10:
                        print(f'{dt} - rainfall={rainfall} mm, humidity={humidity} %')
    except (ValueError, IndexError) as e:
        # Skip problematic rows
        continue

print(f"\nProcessed {len(data)} rainfall/humidity data points")

Processing rainfall and humidity data...

Processed 0 rainfall/humidity data points


In [65]:
# Create CSV file for rainfall and humidity
csv_filename = 'rainfall_processed.csv'

print(f"Creating CSV file: {csv_filename}")

with open(csv_filename, 'w') as f:
    f.write('datetime,rainfall_mm,humidity_pct\n')  # Header
    for record in data:
        dt_str = record[0].strftime("%Y-%m-%d %H:%M")
        rainfall_val = '' if record[1] is None else f'{record[1]:.2f}'
        humidity_val = '' if record[2] is None else f'{record[2]:.1f}'
        f.write(f'{dt_str},{rainfall_val},{humidity_val}\n')

print(f"CSV file created with {len(data)} records")

# Read and display first few lines of the CSV
with open(csv_filename, 'r') as f:
    lines = f.readlines()[:10]
    print("\nFirst 10 lines of CSV:")
    for line in lines:
        print(line.strip())

Creating CSV file: rainfall_processed.csv
CSV file created with 0 records

First 10 lines of CSV:
datetime,rainfall_mm,humidity_pct


## 5. Daily Mean Wind Speed All Year - Kai Tak

We'll fetch the Hong Kong Observatory daily wind CSV and extract the daily mean wind speed for the Kai Tak (KaiTak) station.

The notebook will look for environment variables `WIND_URL` and `WIND_STATION_NAME`. If they are not present, it will fall back to sensible defaults and the previously used `RAINFALL_` variables where appropriate.

In [66]:
# Daily Mean Wind Speed for Kai Tak (from HKO CSV)
# Environment variables: WIND_URL, WIND_STATION_NAME, START_YEAR, END_YEAR
wind_url = os.getenv('WIND_URL') or os.getenv('RAINFALL_URL') or 'https://data.weather.gov.hk/cis/csvfile/SE/ALL/daily_SE_WSPD_ALL.csv'
wind_station = os.getenv('WIND_STATION_NAME') or os.getenv('RAINFALL_STATION_NAME') or 'KaiTak'
start_year = int(os.getenv('START_YEAR', 2010))
end_year = int(os.getenv('END_YEAR', 2025))

print(f"Fetching wind CSV from: {wind_url}")
print(f"Filtering station: {wind_station}")

# Download the CSV (uses caching behavior of get_url if implemented)
csv_filename = 'daily_SE_WSPD_ALL.csv'
csv_text = get_url(wind_url, csv_filename)

# Parse CSV lines
lines = csv_text.splitlines()
header = lines[0] if lines else ''
print(f"CSV header: {header[:200]}")

import csv
reader = csv.DictReader(lines)

# kai_tak_data will be list of dicts: {'date': date_obj, 'mean_wspd': float_or_None, 'station': station_name}
kai_tak_data = []

for row in reader:
    # HKO CSV typically contains a 'station' or 'Station' field and date fields like 'yyyy-mm-dd' or 'date'
    # We'll perform a case-insensitive check for the station name in any of the station-like fields
    station_fields = [k for k in row.keys() if 'station' in k.lower() or 'stn' in k.lower()]
    station_val = None
    for sf in station_fields:
        if row.get(sf):
            station_val = row.get(sf).strip()
            break
    # Some CSVs include station as a separate column, others include station in a combined column; fallback to searching all values
    if station_val is None:
        # look for exact station name anywhere in the row values
        for v in row.values():
            if v and wind_station.lower() in v.lower():
                station_val = v.strip()
                break
    if station_val is None:
        continue
    # Parse date: try common column names
    date_str = None
    for date_key in ['date', 'Date', 'obs_time', 'yyyy-mm-dd', 'yyyymmdd']:
        if date_key in row and row[date_key]:
            date_str = row[date_key].strip()
            break
    if date_str is None:
        # attempt to find a field that looks like a date
        for k, v in row.items():
            if v and re.match(r'\d{4}-\d{2}-\d{2}', v):
                date_str = v.strip()
                break
    if date_str is None:
        continue
    # Parse mean wind speed value: look for 'mean', 'avg', 'wspd' in column names
    wspd_fields = [k for k in row.keys() if any(x in k.lower() for x in ['mean', 'avg', 'wspd', 'w_speed', 'wind'])]
    wspd_val = None
    for wf in wspd_fields:
        v = row.get(wf)
        if v and v.strip() != '':
            # strip non-numeric characters
            num = re.sub(r'[^0-9\.]', '', v)
            if num != '':
                try:
                    wspd_val = float(num)
                    break
                except ValueError:
                    wspd_val = None
    # fallback: check common column names explicitly
    if wspd_val is None:
        for explicit in ['MeanWindSpeed', 'MEAN_WSPD', 'Mean_Wspd', 'WSPD']:
            if explicit in row and row[explicit].strip() != '':
                num = re.sub(r'[^0-9\.]', '', row[explicit])
                if num != '':
                    try:
                        wspd_val = float(num)
                        break
                    except ValueError:
                        wspd_val = None
    # Parse the date into a datetime.date where possible
    try:
        date_obj = datetime.datetime.strptime(date_str[:10], '%Y-%m-%d').date()
    except Exception:
        # skip unparsable dates
        continue
    # Filter by year range
    if date_obj.year < start_year or date_obj.year > end_year:
        continue
    # Match station name loosely (case-insensitive substring)
    if station_val is not None and wind_station.lower() in station_val.lower():
        kai_tak_data.append({'date': date_obj, 'mean_wspd': wspd_val, 'station': station_val})

print(f"Found {len(kai_tak_data)} records for station '{wind_station}' between {start_year} and {end_year}")
if kai_tak_data:
    print('Sample records (first 10):')
    for r in kai_tak_data[:10]:
        print(r)
else:
    print('No matching records found. Check WIND_URL and WIND_STATION_NAME environment variables.')

Fetching wind CSV from: https://data.weather.gov.hk/weatherAPI/cis/csvfile/SE/ALL/daily_SE_RF_ALL.csv
Filtering station: Kaitak
CSV header: ï»¿å¹³åé¢¨é (å


AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
# Prefer the locally generated CSV if it exists to avoid re-downloading the whole HKO file
local_csv = f'kaitak_wind_{start_year}_{end_year}.csv'
kai_tak_data = []
if os.path.exists(local_csv):
    print(f'Loading local processed CSV: {local_csv}')
    import csv
    with open(local_csv, 'r') as f:
        rdr = csv.DictReader(f)
        for row in rdr:
            try:
                d = datetime.datetime.strptime(row['date'], '%Y-%m-%d').date()
            except Exception:
                continue
            val = row.get('mean_wspd')
            wspd = float(val) if val not in (None, '') else None
            kai_tak_data.append({'date': d, 'mean_wspd': wspd, 'station': row.get('station')})
else:
    print('Local processed CSV not found; falling back to parsing the fetched HKO CSV')
    # At this point `reader` and `lines` may be available from the previous cell; if not, re-fetch
    try:
        reader
    except NameError:
        csv_text = get_url(wind_url, 'daily_SE_WSPD_ALL.csv')
        lines = csv_text.splitlines()
        header = lines[0] if lines else ''
        import csv
        reader = csv.DictReader(lines)
    # Fallback parsing: scan DictReader rows for station and wind speed data
    kai_tak_data = []
    for row in reader:
        # attempt to build date and value from common dict fields
        date_str = row.get('date') or row.get('Date') or row.get('yyyy-mm-dd') or ''
        if not date_str:
            # try Year,Month,Day fields
            try:
                y = int(row.get('Year', '0'))
                m = int(row.get('Month', '0'))
                d = int(row.get('Day', '0'))
                date_obj = datetime.date(y, m, d)
            except Exception:
                continue
        else:
            try:
                date_obj = datetime.datetime.strptime(date_str[:10], '%Y-%m-%d').date()
            except Exception:
                continue
        v = row.get('Value') or row.get('value') or row.get('mean_wspd') or ''
        wspd = None
        if v not in (None, ''):
            try:
                wspd = float(re.sub(r'[^0-9\.-]', '', v))
            except Exception:
                wspd = None
        # check station cell
        station_val = row.get('station') or row.get('Station') or ''
        if station_val and wind_station.lower() in station_val.lower():
            kai_tak_data.append({'date': date_obj, 'mean_wspd': wspd, 'station': station_val})

print(f'Loaded kai_tak_data records: {len(kai_tak_data)}')

In [ ]:
# Display summary of Kai Tak daily mean wind speed data
if kai_tak_data:
    print("Summary of Kai Tak wind data:")
    # Show basic stats
    speeds = [r['mean_wspd'] for r in kai_tak_data if r['mean_wspd'] is not None]
    dates = [r['date'] for r in kai_tak_data]
    print(f"Records: {len(kai_tak_data)} (with numeric wind speeds: {len(speeds)})")
    if speeds:
        print(f"Mean wind speed: {sum(speeds)/len(speeds):.2f} (units as in source)")
        print(f"Min: {min(speeds):.2f}, Max: {max(speeds):.2f}")
    print('Sample (first 10)')
    for r in kai_tak_data[:10]:
        print(f"  {r['date']} - {r['station']} - mean_wspd={r['mean_wspd']}")
else:
    print("No Kai Tak wind data was successfully fetched.")
    print("Check WIND_URL, WIND_STATION_NAME, and network connectivity.")

Summary of fetched city data:

City 1: Hong Kong
  Max Temperature: 18.7
  Min Temperature: 14.6
  Rainfall: 33.2

City 2: Lisboa
  Max Temperature: 14.5
  Min Temperature: 8.2
  Rainfall: 109.6

City 3: Porto
  Max Temperature: 13.5
  Min Temperature: 5.1
  Rainfall: 170.8


In [ ]:
# Plot Kai Tak daily mean wind speed (if available)
if 'kai_tak_data' in globals() and kai_tak_data:
    import matplotlib.pyplot as plt
    # Time series (only numeric values)
    numeric = [r for r in kai_tak_data if r['mean_wspd'] is not None]
    if numeric:
        dates = [r['date'] for r in numeric]
        speeds = [r['mean_wspd'] for r in numeric]
        plt.figure(figsize=(12, 4))
        plt.plot(dates, speeds, '-o', markersize=3)
        plt.title('Kai Tak - Daily Mean Wind Speed')
        plt.xlabel('Date')
        plt.ylabel('Mean Wind Speed')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
        # Histogram
        plt.figure(figsize=(8, 4))
        plt.hist(speeds, bins=30, edgecolor='black', alpha=0.7)
        plt.title('Distribution of Daily Mean Wind Speed - Kai Tak')
        plt.xlabel('Mean Wind Speed')
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.show()
    else:
        print('No numeric mean wind speed values to plot')
else:
    print('kai_tak_data not found or empty. Run the previous cell to fetch data first.')

kai_tak_data not found or empty. Run the previous cell to fetch data first.


## 6. Rainfall Data Visualization

Now let's create visualizations of our rainfall data using matplotlib.

In [ ]:
# Prepare data for plotting (rainfall and humidity)
if data:
    plot_data = []  # list of (dt, rainfall_float_or_None, humidity_float_or_None)
    for dt, rainfall, humidity in data:
        # rainfall and humidity are already numeric or None
        plot_data.append((dt, rainfall, humidity))
    
    print(f"Prepared {len(plot_data)} data points for plotting")
    
    # Separate series
    dates = [p[0] for p in plot_data]
    rainfall_values = [p[1] for p in plot_data]
    humidity_values = [p[2] for p in plot_data]
    
    # Compute ranges where data exists
    valid_rain = [v for v in rainfall_values if v is not None]
    valid_hum = [h for h in humidity_values if h is not None]
    if valid_rain:
        print(f"Rainfall range: {min(valid_rain):.2f} mm to {max(valid_rain):.2f} mm")
    if valid_hum:

No rainfall/humidity data available for plotting
